In [28]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np


In [29]:
# load data
data = pd.read_csv("heart_failure.csv")
data.head()

,Unnamed: 0,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,death_event
0,0,75.0,no,582,no,20,yes,265000.00,1.9,130,yes,no,4,1,yes
1,1,55.0,no,7861,no,38,no,263358.03,1.1,136,yes,no,6,1,yes
2,2,65.0,no,146,no,20,no,162000.00,1.3,129,yes,yes,7,1,yes
3,3,50.0,yes,111,no,20,no,210000.00,1.9,137,yes,no,7,1,yes
4,4,65.0,yes,160,yes,20,no,327000.00,2.7,116,no,no,8,1,yes


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                299 non-null    int64  
 1   age                       299 non-null    float64
 2   anaemia                   299 non-null    object 
 3   creatinine_phosphokinase  299 non-null    int64  
 4   diabetes                  299 non-null    object 
 5   ejection_fraction         299 non-null    int64  
 6   high_blood_pressure       299 non-null    object 
 7   platelets                 299 non-null    float64
 8   serum_creatinine          299 non-null    float64
 9   serum_sodium              299 non-null    int64  
 10  sex                       299 non-null    object 
 11  smoking                   299 non-null    object 
 12  time                      299 non-null    int64  
 13  DEATH_EVENT               299 non-null    int64  
 14  death_even

In [31]:
print('Classes and number of values in the dataset',Counter(data['death_event']))

Classes and number of values in the dataset Counter({'no': 203, 'yes': 96})


In [32]:
# select features and labels
x = data[['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time']]
y = data['death_event']

In [33]:
x = pd.get_dummies(x)
x.head()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anaemia_no,anaemia_yes,diabetes_no,diabetes_yes,high_blood_pressure_no,high_blood_pressure_yes,sex_no,sex_yes,smoking_no,smoking_yes
0,75.0,582,20,265000.00,1.9,130,4,1,0,1,0,0,1,0,1,1,0
1,55.0,7861,38,263358.03,1.1,136,6,1,0,1,0,1,0,0,1,1,0
2,65.0,146,20,162000.00,1.3,129,7,1,0,1,0,1,0,0,1,0,1
3,50.0,111,20,210000.00,1.9,137,7,0,1,1,0,1,0,0,1,1,0
4,65.0,160,20,327000.00,2.7,116,8,0,1,0,1,1,0,1,0,1,0


In [34]:
# split into train test sets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [36]:
# standardize data
# ct = ColumnTransformer([("numeric", StandardScaler(), ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time'])])

numerical_features = x.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

In [37]:
# transform data
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [39]:
# initialize label encoder
le = LabelEncoder()

Y_train = le.fit_transform(Y_train.astype(str))
Y_test = le.transform(Y_test.astype(str))

In [40]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [43]:
# instantiate model
model = Sequential()

# create input layer
model.add(InputLayer(input_shape=(X_train.shape[1],)))

# create hidden layer
model.add(Dense(12, activation='relu'))

# create output layer
model.add(Dense(2, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
# fit model to data
model.fit(X_train, Y_train, epochs = 100, batch_size = 16, verbose=1)

# calc acc
loss, acc = model.evaluate(X_test, Y_test, verbose=0)
print("Loss", loss, "Accuracy:", acc)

Epoch 1/100
14/14 [==============================] - 0s 1ms/step - loss: 0.7908 - accuracy: 0.3971
Epoch 2/100
14/14 [==============================] - 0s 691us/step - loss: 0.7086 - accuracy: 0.4498
Epoch 3/100
14/14 [==============================] - 0s 752us/step - loss: 0.6448 - accuracy: 0.6124
Epoch 4/100
14/14 [==============================] - 0s 976us/step - loss: 0.6034 - accuracy: 0.7081
Epoch 5/100
14/14 [==============================] - 0s 844us/step - loss: 0.5687 - accuracy: 0.7608
Epoch 6/100
14/14 [==============================] - 0s 921us/step - loss: 0.5459 - accuracy: 0.7656
Epoch 7/100
14/14 [==============================] - 0s 845us/step - loss: 0.5254 - accuracy: 0.7703
Epoch 8/100
14/14 [==============================] - 0s 719us/step - loss: 0.5073 - accuracy: 0.7751
Epoch 9/100
14/14 [==============================] - 0s 769us/step - loss: 0.4884 - accuracy: 0.7799
Epoch 10/100
14/14 [==============================] - 0s 1ms/step - loss: 0.4734 - accuracy: 

14/14 [==============================] - 0s 841us/step - loss: 0.2924 - accuracy: 0.8756
Epoch 82/100
14/14 [==============================] - 0s 765us/step - loss: 0.2913 - accuracy: 0.8804
Epoch 83/100
14/14 [==============================] - 0s 614us/step - loss: 0.2906 - accuracy: 0.8756
Epoch 84/100
14/14 [==============================] - 0s 767us/step - loss: 0.2915 - accuracy: 0.8756
Epoch 85/100
14/14 [==============================] - 0s 844us/step - loss: 0.2902 - accuracy: 0.8756
Epoch 86/100
14/14 [==============================] - 0s 691us/step - loss: 0.2889 - accuracy: 0.8756
Epoch 87/100
14/14 [==============================] - 0s 844us/step - loss: 0.2882 - accuracy: 0.8756
Epoch 88/100
14/14 [==============================] - 0s 918us/step - loss: 0.2879 - accuracy: 0.8756
Epoch 89/100
14/14 [==============================] - 0s 612us/step - loss: 0.2865 - accuracy: 0.8756
Epoch 90/100
14/14 [==============================] - 0s 587us/step - loss: 0.2858 - accuracy: 

In [49]:
# classification report
y_estimate = model.predict(X_test, verbose=0)
y_estimate = np.argmax(y_estimate, axis = -1)
y_true = np.argmax(Y_test, axis=1)
print(classification_report(y_true, y_estimate))

              precision    recall  f1-score   support

           0       0.83      0.95      0.89        62
           1       0.84      0.57      0.68        28

    accuracy                           0.83        90
   macro avg       0.84      0.76      0.78        90
weighted avg       0.83      0.83      0.82        90

